In [1]:
!pip install pynmea2

In [45]:

import numpy as np
import struct #https://docs.python.org/3/library/struct.html
import pynmea2
import pandas as pd
from datetime import datetime, timedelta #for handleing time
import re  #for string parcing
import os #for path search

INPUT DATA: FILES NAMES, SCALES FOR GARMIN GPS RESOLUTION

In [109]:
folder_raw_data='data_raw'#'20230223_caminata_pruebas_rf.DAT'\
filename_raw_data='20230224_TORTO006_RF0_tortuga185'#'20230223_caminata_pruebas_rf.DAT'\
filename_folder_data_out='data_csv'#'20230223_caminata_pruebas_rf.DAT'\
filename= folder_raw_data+'/'+filename_raw_data+'.DAT'
print(filename)
#if you are using a manual gps to track animal position and wish to compare data, fill the following:
filename_gpsgarmin='CalibSAO2.csv'
gps_garmin_hdop_circle_size=5 #change to make the visual part according
gps_decimal_to_meters=111139#convert decimal to meters. 111139 meters equals 1.0 lat/lon value in decimal
gps_garmin_lat_error=1/gps_decimal_to_meters #Convert decimal to meters, uANDRES:added for completing the code, i do not know the real value of this
gps_garmin_lon_error=1/gps_decimal_to_meters #:added for completing the code, add the real value


data_raw/TORTO001V1.1.DAT


READ THE DATA, CONVERT IT FROM BINARY TO DECIMAL VALUES

In [122]:
UNSIGNED_INT='<I' #4 BYTE UNSIGNED INT
SIGNED_INT='<i' #4 BYTE UNSIGNED INT
CHAR_TASK_NAME='12s' #4 BYTE UNSIGNED INT
CHAR_TASK_NAME_OFFSET=2
CHAR_TASK_NAME_LEN=12
CHAR_TASK_NAME_BOOTUP_WRITE=b'SYSTEM INFO' #4 BYTE UNSIGNED INT
DATA_SIZE_OFFSET=14
DATA_OFFSET=16
DATA_TIME_RES_OFFSET=244 #V1.0 is this value
RTOS_time_resolutionHz=1000 #default value
#get time value
TIME_OFFSET=20 #lenght=4 byutes
#
GPS_STRUCT='85s' #
GPS_S_SIZE_MESS=85  #96 ORI
GPS_S_SIZE=104  #96 ORI
GPS_S_START=41 #
GPS_S_END=GPS_S_START+85   #114 ORI
GPS_S_TFIX=GPS_S_START+GPS_S_SIZE_MESS+10  #TIME TO FIX OFFSET REFFERED TO THE END OF MESSAGE
GPS_S_START_T=GPS_S_END+2#st
GYRO_START=50#50 is the value
MAG_START=40#40 is the real value
TEMPERATURE_SIZE=512
ANIM_STRUCT='76s'#
ANIM_SIZE=76#
dt = np.dtype(np.uint8)      # 8-bit unsigned integer
#CR=13
#LF=10
time_start=0;
start_time_raw=0
end_time_raw=0
np.set_printoptions(precision=3) #print 3 digits only
acc_scaler= 0.00059841#scale 0.000061=1g, 0,00059841=1 m/s
gyro_scaler= 0.00875  #scale 0.00875 for +-245 degree
mag_scaler=0.00014 #scale: 0.00014 for +-1T
#global variables
global float_res_time;
float_res_time=3;#default value
#return the indexes in which an structure begin
def get_time_bootup(byte_array):
      fr_version=bytes.decode((byte_array[215:218]).tobytes(), 'ascii')
      print(fr_version)
      if fr_version=='1.0':
          RTOS_time_resolutionHz=bytes_to_32uint(byte_array[244:244+4])
          dateTime_rtos_offset_secs=(bytes_to_32uint(byte_array[260:260+4])/RTOS_time_resolutionHz)
          str_toparse_dateTimeUTC=bytes.decode((byte_array[110:180]).tobytes(), 'ascii')
          str_aux=str_toparse_dateTimeUTC.split('date:')[1]
          str_aux_date=str_aux.split(' ')[0]
          str_aux2=str_toparse_dateTimeUTC.split('time ')[1]
          str_aux_time=str_aux2.split(' UTC')[0]
          #get date time
          dateTime_UTC_str=str_aux[0:4]+'-'+str_aux[4:6]+'-'+str_aux[6:8]+' '+str_aux_time
      else:#version 1.1 and default
          RTOS_time_resolutionHz=bytes_to_32uint(byte_array[244:244+4]) #same as 1.0
          dateTime_rtos_offset_secs=(bytes_to_32uint(byte_array[268:268+4])/RTOS_time_resolutionHz)
          print("date time offset",dateTime_rtos_offset_secs)
          str_toparse_dateTimeUTC=bytes.decode((byte_array[110:180]).tobytes(), 'ascii')  
          str_aux=str_toparse_dateTimeUTC.split('date:')[1]
          dateTime_UTC_str=str_aux.split(' UTC')[0]
      print(dateTime_UTC_str)
      dateTime_UTC=datetime.strptime(dateTime_UTC_str, '%Y-%m-%d %H:%M:%S')#does not include decimals
      print(dateTime_UTC)
      return RTOS_time_resolutionHz,dateTime_UTC,dateTime_rtos_offset_secs
def get_animal_act(byte_array,dateTime_UTC,last_act_time,last_act,time_start):
      annim_str=(struct.unpack(ANIM_STRUCT, byte_array[0:ANIM_SIZE]) [0]).decode('UTF-8')#="!NEW ANIMAL ACTIVITY DETECTED, CURRENT STATE: 1, at time: 5.115 [sec]"
      animal_state=re.findall(r"[-+]?(?:\d*\.*\d+)",annim_str)
      animal_act=int(animal_state[0])
      time=time_start+float(animal_state[1])
      act_ret=[];time_ret=[];  
      act_ret.append(last_act); #double append
      act_ret.append(animal_act); #double append
      f_res_time=-6+1#l->-6+1->1 digits for time only
      #print(act_ret)      #print(animal_act,animal_state)
      time_add= dateTime_UTC+ timedelta(seconds=time)
      time_ret.append(time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      time_ret.append(time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time]);
      return act_ret,time_ret
def get_temperature(byte_array,dateTime_UTC,time_start,time_end):
      global float_res_time;
      t_mcu=[];t_imu=[];time=[];
      data_written=bytes_to_16uint(byte_array[DATA_OFFSET-1:DATA_OFFSET+1])
      time_res=(time_end-time_start)/data_written 
      #print(data_written,time_res) 
      TEMP_START=32
      data_array_size=8
      f_res_time=-6+1#low sample rate=low resolution->-6+1->1 digit for time only
      for i in range(data_written):
            print(i)
            c_pos=TEMP_START+data_array_size*i
            t_imu.append(round(bytes_to_float(byte_array[c_pos:c_pos+4]), 1) ) #1 decimal
            t_mcu.append(bytes_to_16int(byte_array[c_pos+4:c_pos+4+2]))
            time_add= dateTime_UTC+ timedelta(seconds=time_start+i*time_res)
            time.append(time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      print(t_mcu)
      return time,t_mcu,t_imu
def get_mag(byte_array,dateTime_UTC,time_start,time_end):
      global float_res_time;
      mx=[];my=[];mz=[];time=[];
      data_written=bytes_to_16uint(byte_array[DATA_OFFSET:DATA_OFFSET+2])
      time_res=(time_end-time_start)/data_written 
      #print(time_res)
      float_digits=4
      f_res_time=-6+2 #low sample rate=low resolution->-6+2->2 digits foir time only
      data_array_size=6
      for i in range(data_written):
            c_pos=MAG_START+data_array_size*i
            mx.append(round(bytes_to_mag(byte_array[c_pos:c_pos+2]),float_digits))
            my.append(round(bytes_to_mag(byte_array[c_pos+2:c_pos+2+2]),float_digits))
            mz.append(round(bytes_to_mag(byte_array[c_pos+4:c_pos+4+2]),float_digits))
            time_add= dateTime_UTC+ timedelta(seconds=time_start+i*time_res)
            time.append(time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      return mx,my,mz,time
def get_accelGyro(byte_array,dateTime_UTC,time_start,time_end):
      global float_res_time;
      gx=[];gy=[];gz=[]; ax=[];ay=[];az=[];a_time=[];
      data_written=bytes_to_16uint(byte_array[DATA_OFFSET:DATA_OFFSET+2])
      time_res=(time_end-time_start)/data_written
      float_digitsg=2
      float_digitsa=4
      f_res_time=-6+4
      gyro_on=bytes_to_16uint(byte_array[34:36]) #DEFINED
      print(time_start)  
      print(time_res)  
      if gyro_on:
        for i in range(data_written):
            c_pos=GYRO_START+12*i
            gx.append(round(bytes_to_gyro(byte_array[c_pos:      c_pos+2]),float_digitsg))
            gy.append(round(bytes_to_gyro(byte_array[c_pos+2:  c_pos+2+2]),float_digitsg))
            gz.append(round(bytes_to_gyro(byte_array[c_pos+4:  c_pos+4+2]),float_digitsg))
            ax.append(round(bytes_to_acc(byte_array[c_pos+6:  c_pos+6+2]),float_digitsa))
            ay.append(round(bytes_to_acc(byte_array[c_pos+8:  c_pos+8+2]),float_digitsa))
            az.append(round(bytes_to_acc( byte_array[c_pos+10:c_pos+10+2]),float_digitsa))
            a_time_add= dateTime_UTC+ timedelta(seconds=time_start+i*time_res)
            a_time.append(a_time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      else:
        for i in range(data_written): 
            c_pos=GYRO_START+6*i
            gx.append(0) #append a 0 to maintaain consystency
            gy.append(0)
            gz.append(0)
            ax.append(round(bytes_to_acc(byte_array[c_pos:    c_pos+2]),float_digitsa))
            ay.append(round(bytes_to_acc(byte_array[c_pos+2:c_pos+2+2]),float_digitsa))
            az.append(round(bytes_to_acc(byte_array[c_pos+4:c_pos+4+2]),float_digitsa))
            a_time_add= dateTime_UTC+ timedelta(seconds=time_start+i*time_res)
            a_time.append(a_time_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:f_res_time])
      print(a_time)
      return gx,gy,gz,ax,ay,az,a_time
def get_gpsneoxm(byte_array,dateTime_UTC,time_start,time_end,time_scaler):
      global float_res_time;
      lat=[];lon=[];vdop=[];hdop=[];pdop=[];time_gps=[];time_gps_to_fix=[];
      for i in range(6):
            # print(i)
             gps_msg=struct.unpack(GPS_STRUCT,byte_array[GPS_S_SIZE*i+GPS_S_START:GPS_S_SIZE*i+GPS_S_END])[0]
             gps_timefix_raw=struct.unpack(UNSIGNED_INT,byte_array[GPS_S_SIZE*i+GPS_S_TFIX:GPS_S_SIZE*i+GPS_S_TFIX+4])[0]     
            #print(gps_msg.decode('UTF-8'))
             if (byte_array[GPS_S_START])!=ord(' '):#character DIFERENT FROM WHITESPACE->>parse
                 gps = pynmea2.parse(gps_msg.decode('UTF-8'))
                 gps_time=round(time_start+gps_timefix_raw*time_scaler,1)
                 gps_time_to_fix="{:.1f}".format((gps_timefix_raw*time_scaler))#only 1 decimal
                 time_gps_add = dateTime_UTC+ timedelta(seconds=gps_time)
                 gps_type=gps.sentence_type
                 if gps_type=='RMC':
                     time_gps.append(time_gps_add.strftime("%Y-%m-%d %H:%M:%S.%f")[:-5])#append in format. [:-5]=removes 5 decimals in useconds
                     time_gps_to_fix.append(float(gps_time_to_fix))
                     lat.append(round(gps.latitude,6))
                     lon.append(round(gps.longitude,6))
                    # print(gps.sentence_type,"lat ","{:.4f}".format(gps.latitude), "long","{:.4f}".format(gps.longitude),'at ',gps_time,'[s]')
                 if gps_type=='GSA':      
                    vdop.append(round(float(gps.vdop),1))
                    hdop.append(round(float(gps.hdop),1))           
                    pdop.append(round(float(gps.pdop),1)) 
                    #print(gps.sentence_type,'hdop',gps.hdop,'vdop',gps.vdop,'pdop',gps.hdop,'at ',gps_time,'[s]')
      return(lat,lon,vdop,hdop,pdop,time_gps,time_gps_to_fix)    
#auxiliwary functions
def bytes_to_float(byte_array):
    return(struct.unpack('f', byte_array) [0])
def bytes_to_32uint(byte_array):
     return struct.unpack(UNSIGNED_INT, byte_array)[0]
     # return (byte_array[0]+256*byte_array[1]+256*byte_array[1]+65534*byte_array[2]+16777216*byte_array[3])
def bytes_to_16uint(byte_array):
     return(byte_array[0]+256*byte_array[1])
def bytes_to_16int(byte_array):
     signed=bytes_to_16uint(byte_array)
     if signed>32767: signed=signed-65536
     return(signed)
def bytes_to_8int(byte_array):
      signed=byte_array[0]
      if signed>127: signed=signed-255
      return(signed)
def bytes_to_mag(byte_array):
      return(mag_scaler*bytes_to_16int(byte_array))
def bytes_to_acc(byte_array):
      return(acc_scaler*bytes_to_16int(byte_array))
def bytes_to_gyro(byte_array):
      return(gyro_scaler*bytes_to_16int(byte_array))
def find_start_indexes(byte_array):
    index_with_lines_br = np. where((byte_array == 13 ))[0]
    intex_ok=[]
    for index in index_with_lines_br:
        #print(index)
        if ((byte_array[index+1]==10)and((123>byte_array[index+2]>47)or (byte_array[index+2]==33))):
            intex_ok.append(index)
    return intex_ok
def get_datacsv_from_file(byte_array,intexes_ok):
     global float_res_time;
     lat=[];lon=[];vdop=[];hdop=[];pdop=[];time_gps=[];time_gps_to_fix=[];
     gx=[];gy=[];gz=[];ax=[];ay=[];az=[];a_time=[];
     mx=[];my=[];mz=[];m_time=[];
     t_mcu=[];t_imu=[];t_time=[];
     animal_activity=[];animal_time=[];
     time_start=0;
     start_time_raw=0
     end_time_raw=0
     for index in intexes_ok:
       task_name=struct.unpack(CHAR_TASK_NAME, byte_array[index+CHAR_TASK_NAME_OFFSET:index+CHAR_TASK_NAME_OFFSET+CHAR_TASK_NAME_LEN]) [0]
       print(task_name[0:12])
       if task_name[0:len(CHAR_TASK_NAME_BOOTUP_WRITE)]==CHAR_TASK_NAME_BOOTUP_WRITE:#detect if a write has happened
           #GET OPERATIVE SYSTEM TIME RESOLUTION AND DATE FOR FUTURE READINGS.
           RTOS_time_resolutionHz,dateTime_UTC,dateTime_rtos_offset_secs=get_time_bootup(byte_array[index:index+300])
           if (RTOS_time_resolutionHz<1001): float_res_time=3 #ms resolution
           else:                                              float_res_time=6 #us resolution
           time_scaler=round(1/RTOS_time_resolutionHz,float_res_time) #convert hz to sec with desired resolution
           print("first write time resolution in ms:",time_scaler)
           time_start=start_time_raw#Error: No such resource: /ti/utils/runtime/ExecGraphAdapter.syscfg.js#fix using the last started time
           print("System boot up at:",dateTime_UTC)
           if len(animal_time)==0:
                animal_time.append(dateTime_UTC.strftime("%Y-%m-%d %H:%M:%S"))
                animal_activity.append(0)
       else: 
           #parse as timestamp of sensor data: this means that
           data_size=bytes_to_16uint(byte_array[index+DATA_SIZE_OFFSET:index+DATA_SIZE_OFFSET+2])
           data_written=bytes_to_16uint(byte_array[index+DATA_OFFSET:index+DATA_OFFSET+2])
           # -dateTime_rtos_offset_secs is the RTOS time in which the dateTime_UTC was stored.
           start_time_raw=time_start-dateTime_rtos_offset_secs+struct.unpack(UNSIGNED_INT, byte_array[index+TIME_OFFSET:index+TIME_OFFSET+4])[0] #convert 4 bytes to unisgned int
           end_time_raw=time_start-dateTime_rtos_offset_secs+struct.unpack(UNSIGNED_INT, byte_array[index+TIME_OFFSET+4:index+TIME_OFFSET+8])[0] #convert 4 bytes to unisgned int
           start_time_offset=start_time_raw*time_scaler
           end_time_offset=end_time_raw*time_scaler
    #start to parse the packet 
       #print((task_name.decode('UTF-8')), 'from',"{:.2f}".format(start_time_raw*time_scaler),"[s] to ","{:.2f}format(end_time_raw*time_scaler),'[s]')
       if(task_name[0:6]==b'IMU0AG'):##accelerometer/gyro
            gxu,gyu,gzu,axu,ayu,azu,a_timeu=get_accelGyro(byte_array[index:index+data_size],dateTime_UTC,start_time_offset,end_time_offset)
            gx=gx+(gxu);gy=gy+(gyu);gz=gz+(gzu);ax=ax+(axu);ay=ay+(ayu);az=az+(azu);a_time=a_time+(a_timeu);
           # print(index,"G/Adata_written",data_written,bytes_to_acc(byte_array[index+GYRO_START+10:index+GYRO_START+10+2]))
       if(task_name[0:5]==b'IMU0M'):#magnetometer
            mxu,myu,mzu,m_timeu=get_mag(byte_array[index:index+data_size],dateTime_UTC,start_time_offset,end_time_offset)
            mx=mx+(mxu);my=my+(myu);mz=mz+(mzu);m_time=m_time+(m_timeu);
           # print(index,"Mdata_written",data_written)
       if (task_name[0:10]==b'TEMPERATURE'):#temperature
            print(index,byte_array[index:index+24])
            t_timeu, t_mcuu, t_imuu=get_temperature(byte_array[index:index+data_size],dateTime_UTC,start_time_offset,end_time_offset)
            t_time=t_time+t_timeu;t_mcu=t_mcu+(t_mcuu);t_imu=t_imu+(t_imuu);
       if(task_name[0:11]==b'!NEW ANIMAL'):
            animal_act,animal_t=get_animal_act(byte_array[index:index+data_size],dateTime_UTC,animal_time[-1],animal_activity[-1],time_start*time_scaler)
            animal_activity=animal_activity+(animal_act);animal_time=animal_time+(animal_t);
            #print(animal_sta,animal_t)
       if(task_name[0:8]==b'GPSNEO7M'):#work ok, detect gps 
           lat,lon,vdop,hdop,pdop,time_gps,time_gps_to_fix=get_gpsneoxm(byte_array[index:index+data_size],dateTime_UTC,start_time_offset,end_time_offset,time_scaler);
    #----------accel gyro
     data=np.concatenate([np.array(a_time),np.asarray(ax),np.asarray(ay),np.asarray(az),np.asarray(gx),np.asarray(gy),np.asarray(gz)]);
     data=data.reshape(7,(int(len(data)/7)))
     accel_frame= pd.DataFrame(data.T,columns=['dateTime_UTC','ax', 'ay', 'az','gx', 'gy', 'gz'])
     #-----------mag   
     data=np.concatenate([np.array(m_time),np.asarray(mx),np.asarray(my),np.asarray(mz)]);
     data=data.reshape(4,(int(len(data)/4)))
     mag_frame= pd.DataFrame(data.T,columns=['dateTime_UTC','mx', 'my', 'mz']) 
      #----------temperature 
     data=np.concatenate([np.array(t_time),np.asarray(t_mcu),np.asarray(t_imu)]);
     data=data.reshape(3,(int(len(data)/3)))
     temperature_frame= pd.DataFrame(data.T,columns=['dateTime_UTC','temp_mcu', 'temp_imu']) 
     #---------animal state
     #print((animal_activity))
     #print((animal_time))
     data=np.concatenate([np.array(animal_time),np.asarray(animal_activity)]);
     data=data.reshape(2,(int(len(data)/2)))
     animal_activity_frame= pd.DataFrame(data.T,columns=['dateTime_UTC','animal_activity']) 
     #---------gps   
     data=np.concatenate([np.array(time_gps),np.array(time_gps_to_fix),np.asarray(lat),np.asarray(lon),np.asarray(vdop),np.asarray(hdop),np.asarray(pdop)]);
     data=data.reshape(7,(int(len(data)/7)))
     gps_frame= pd.DataFrame(data.T,columns=['dateTime_UTC','time_to_fix','lat', 'lon', 'vdop','hdop','pdop'])
     return(accel_frame,mag_frame,temperature_frame,gps_frame,animal_activity_frame)

byte_array = np.fromfile(filename, dtype=dt)

print("file size in bytes:",byte_array.size)
intexes_ok=find_start_indexes(byte_array)
print("indices ok",intexes_ok[0:10])
accel_frame,mag_frame,temperature_frame,gps_frame,animal_activity_frame=get_datacsv_from_file(byte_array,intexes_ok)
#export to csv
#-------------------------------------------------------------------
#check if output dir exist, otherwise, create it 
if not os.path.exists(filename_folder_data_out):
    os.makedirs(filename_folder_data_out)
#
filename_data_out=filename_folder_data_out+'/'+filename_raw_data
gps_frame.to_csv(filename_data_out+'_gps.csv',index=False)
accel_frame.to_csv(filename_data_out+'_accelGyro.csv',index=False)
mag_frame.to_csv(filename_data_out+'_mag.csv',index=False)
temperature_frame.to_csv(filename_data_out+'_temperature.csv',index=False)
animal_activity_frame.to_csv(filename_data_out+'_animal_activity.csv',index=False)
print("FINISH TO CONVERT DATA INTO CSV",intexes_ok[0:10])

file size in bytes: 44598
indices ok [0, 296, 386, 476, 1500, 5596, 7644, 11740, 15836, 17884]
b'SYSTEM INFO\x00'
  1
date time offset 0.0615
2023-06-17 01:25:31
2023-06-17 01:25:31
first write time resolution in ms: 0.0005
System boot up at: 2023-06-17 01:25:31
b'!NEW ANIMAL '
b'!NEW ANIMAL '
b'GPSNEO7M\x00\x00\x00\x00'
b'IMU0AG\x00\x00\x00\x00\x00\x00'
23.74396925
0.05882302405498284
['2023-06-17 01:25:54.7439', '2023-06-17 01:25:54.8027', '2023-06-17 01:25:54.8616', '2023-06-17 01:25:54.9204', '2023-06-17 01:25:54.9792', '2023-06-17 01:25:55.0380', '2023-06-17 01:25:55.0969', '2023-06-17 01:25:55.1557', '2023-06-17 01:25:55.2145', '2023-06-17 01:25:55.2733', '2023-06-17 01:25:55.3321', '2023-06-17 01:25:55.3910', '2023-06-17 01:25:55.4498', '2023-06-17 01:25:55.5086', '2023-06-17 01:25:55.5674', '2023-06-17 01:25:55.6263', '2023-06-17 01:25:55.6851', '2023-06-17 01:25:55.7439', '2023-06-17 01:25:55.8027', '2023-06-17 01:25:55.8616', '2023-06-17 01:25:55.9204', '2023-06-17 01:25:55.9

In [123]:
import plotly.graph_objects as go
import pandas as pd
filename_data_out=filename_folder_data_out+'/'+filename_raw_data
gps_frame=pd.read_csv(filename_data_out+'_gps.csv')
accel_frame=pd.read_csv(filename_data_out+'_accelGyro.csv')
mag_frame=pd.read_csv(filename_data_out+'_mag.csv')
temperature_frame=pd.read_csv(filename_data_out+'_temperature.csv')
animal_activity_frame=pd.read_csv(filename_data_out+'_animal_activity.csv')
#-----------------accel
figa=go.Figure()#plot acceleration in ratio 9.81 m/s*2 =1
figa.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['ax'].values[:],name="ax",line_width=3))#px.scatter(accel_frame, x="time", y="ax")
figa.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['ay'].values[:],name="ay",line_width=3))
figa.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['az'].values[:],name="az",line_width=3))
figa.update_layout(title="Accelemerometer meassures ",xaxis_title="Time [Seconds]",yaxis_title="Accelemerometer axis value [m/s²]",legend_xanchor="right")
figa.show()
#-----------------animal_activity
fig_activity=go.Figure()#plot acceleration in ratio 9.81 m/s*2 =1
fig_activity.add_trace(go.Scatter( x=animal_activity_frame['dateTime_UTC'].values[:], y=animal_activity_frame['animal_activity'].values[:],name="activity",line_width=5))#px.scatter(accel_frame, x="time", y="ax")
fig_activity.update_layout(title="Animal Activity. ",xaxis_title="Time [Seconds]",yaxis_title="1: still, 2: moving ",legend_xanchor="right")
fig_activity.show()
#-----------------gyro
figg=go.Figure()#plot gyroscope in degree/s
figg.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['gx'].values[:],name="gx",line_width=3))#px.scatter(accel_frame, x="time", y="ax")
figg.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['gy'].values[:],name="gy",line_width=3))
figg.add_trace(go.Scatter( x=accel_frame['dateTime_UTC'].values[:], y=accel_frame['gz'].values[:],name="gz",line_width=3))
figg.update_layout(title="Gyroscope meassures ",xaxis_title="Time [Seconds]",yaxis_title="Gyroscope axis value [°/s]",legend_xanchor="right")
figg.show()
#-----------------mag
figmag=go.Figure()#plot acceleration in ratio 9.81 m/s*2 =1
figmag.add_trace(go.Scatter( x=mag_frame['dateTime_UTC'].values[:], y=mag_frame['mx'].values[:],name="mx",line_width=3))#px.scatter(accel_frame, x="time", y="ax")
figmag.add_trace(go.Scatter( x=mag_frame['dateTime_UTC'].values[:], y=mag_frame['my'].values[:],name="my",line_width=3))
figmag.add_trace(go.Scatter( x=mag_frame['dateTime_UTC'].values[:], y=mag_frame['mz'].values[:],name="mz",line_width=3))
figmag.update_layout(title="Magnetomer meassures ",xaxis_title="Time [Seconds]",yaxis_title="Magnetometer axis value [T]",legend_xanchor="right")
figmag.show()
#-----------------Temperature
fig_temperature=go.Figure()#plot acceleration in ratio 9.81 m/s*2 =1
fig_temperature.add_trace(go.Scatter( x=temperature_frame['dateTime_UTC'].values[:], y=temperature_frame['temp_mcu'].values[:],name="Temperature MCU",line_width=5))#px.scatter(accel_frame, x="time", y="ax")
fig_temperature.add_trace(go.Scatter( x=temperature_frame['dateTime_UTC'].values[:], y=temperature_frame['temp_imu'].values[:],name="Temperature IMU",line_width=5))#px.scatter(accel_frame, x="time", y="ax")
fig_temperature.update_layout(title="Temperature meassures ",xaxis_title="Time [Seconds]",yaxis_title=" Degrees C",legend_xanchor="right")
fig_temperature.show()
#---------------------gps histogram
import plotly.express as px
fig = px.histogram(gps_frame, x="time_to_fix")
fig.show()
#--------------- GPS PLOT
import plotly.express as px
scale_gpsplot=7# px.scatter_mapbox(gps_frame, lat='lat', lon='lon', color="hdop",hover_name="time", size='hdop',
figgps =px.scatter_mapbox(gps_frame, lat='lat', lon='lon',hover_name='dateTime_UTC',
                  color_continuous_scale=px.colors.sequential.Sunsetdark, size_max=20, zoom=18,                 
                  mapbox_style="carto-positron") #Sunsetdark
figgps_line = px.line_mapbox(gps_frame, lat='lat', lon='lon',hover_name='dateTime_UTC',
                   zoom=18,
                    mapbox_style="carto-positron" ) #Sunsetdark
figgps.add_trace(figgps_line.data[0]) # adds the line trace to the first figure
figgps.show()


In [ ]:
#expoort plots as interactives html
figa.write_html(filename_data_out+"_accel.html")
figg.write_html(filename_data_out+"_gyro.html")
figmag.write_html(filename_data_out+"_mag.html")
fig_activity.write_html(filename_data_out+"_activty.html")
fig_temperature.write_html(filename_data_out+"_temperature.html")
figgps.write_html(filename_data_out+"_gps.html")

In [ ]:
#detect activity
activity_gap=299
gap_treshhold=30
def get_activity_gap(data_pd,gap_treshhold,activity_gap):
      time_list=[]
      activity_valid=True
      time_gap=0
      last_time_gap=0
      for index, row in data_pd.iterrows():
              if row['gx']>gap_treshhold:
                     time_gap=row['time']
                     if time_gap>(last_time_gap+activity_gap):
                         time_list.append(time_gap)
                         last_time_gap=time_gap
                     #long.append(gps.longitude)
                    # print(gps.sentence_type,"lat ","{:.4f}".format(gps.latitude), "long","{:.4f}".format(gps.longitude),'at ',gps_time,'[s]')
      return(time_list)  
    #gps_frame= pd.DataFrame(data.T,columns=['time','lat', 'long', 'vdop','hdop','pdop'])
        #print (index,row["Fee"], row["Courses"])
def keep_cols(DataFrame, keep_these):
    """Keep only the columns [keep_these] in a DataFrame, deleteall other columns. 
    """
    DataFrame_2=DataFrame
    drop_these = list(set((DataFrame_2.index)) - set(keep_these))
    #print(drop_these)
    return DataFrame_2.drop(drop_these)
def get_gps_at_time(data_pd,gps_times):
      """returns only the first gps postion meassures for each gps_times
      """
      time_list=[]
      activity_valid=True
      time_gap=0
      last_time_gap=0
      data_pd_out=data_pd
      row_list=[]  
      for time in gps_times:
              last_time=0
              for index, row in data_pd.iterrows(): 
                    if last_time==0:
                        if row['time']>=time:
                           last_time=1
                           row_list.append(index) 
      return(row_list,keep_cols(data_pd,row_list))

def gps_get_metrics(data_pd):
        """get metrics of gps
        """
        df = pd.DataFrame( {"to_drop": [0]})#, columns = ['to_drop'], index = ['a'])
        if len(data_pd)==0:is_data_to_store=False
        else: is_data_to_store=True
        #store data if there is data to store    
        if is_data_to_store: 
            df['time']=data_pd['time'].values[0]#store the first time
            columns=['lat','lon','hdop','pdop']
            data=[]
            for col in columns:
                name_std=col+' std'
                median=data_pd[col].median()
                std=data_pd[col].std()
                df[col]=median
                df[name_std]=std
                #asign to data frame
                #data_pd[name_std] = abs( data_pd[col] - median )**(0.5) #this is to get std for each column
        print(df)
        df=df.drop("to_drop",axis='columns')
       # print(df)
        return(df)


def get_gps_at_time_metrics(data_pd,gps_times):
      """returns only the average and std of gps postion meassures for each gps_times
      """
      activity_valid=True
      last_time_gap=0
      data_pd_out=data_pd
      last_row=0
      return_pd=pd.DataFrame()
     # print(gps_times)
      last_index=0
      time_i_len=len(data_pd['time'].values)
      for time_i in range(len(gps_times)):
      #do this for each time
              last_time=0
              row_list=[] 
             # print('last index',last_index)
              for index in range(last_index,time_i_len):
                    if last_time==0:
                        time=data_pd['time'].values[index]
                        #print(time,gps_times[time_i])
                        if time<=gps_times[time_i] :
                           row_list.append(index)
                        if time_i<(time_i_len):
                         if time>gps_times[time_i]:
                           last_time=1 #finish to iterate
                           last_index=index
                           #print('last index got',last_index)
                           break
              #at this point we have all the indexes of data, so we proceed to the avergage
              aux_pd=keep_cols(data_pd,row_list)
              #print(row_list)
              pd_got=gps_get_metrics(aux_pd)
                
              if (len(row_list))>0:return_pd=pd.concat([return_pd,pd_got] ) 
                #return_pd.append(gps_get_metrics(aux_pd), ignore_index = False)
                  #long.append(gps.longitude)
                     # print(gps.sentence_type,"lat ","{:.4f}".format(gps.latitude), "long","{:.4f}".format(gps.longitude),'at ',gps_time,'[s]')
      return(return_pd)  
    #gps_frame= pd.DataFrame(data.T,columns=['time','lat', 'long', 'vdop','hdop','pdop'])
#---------------main program    
# get the times in which a movement event happended
gps_times=get_activity_gap(accel_frame,gap_treshhold,activity_gap)
#return the list of r
#row_list,gps_frame_edited=get_gps_at_time(gps_frame,gps_times)
gps_degree_to_meters=111139#convert degrees to meters
#gps_frame_edited=get_gps_at_time_metrics(gps_frame,gps_times)
gps_frame_edited=gps_get_metrics(gps_frame)
gps_frame_edited['dist std [m]'] =gps_degree_to_meters* ( gps_frame_edited['lat std']**2+ gps_frame_edited['lon std']**2 )**(0.5) #this is to get std for each column
print(gps_frame_edited)
figgps_edit = px.scatter_mapbox(gps_frame_edited, lat='lat', lon='lon', color="hdop",hover_name="time", size='dist std [m]',
                  color_continuous_scale=px.colors.sequential.Sunsetdark, size_max=20, zoom=18,
                  mapbox_style="carto-positron") #Sunsetdark
figgps_edit_line =  px.line_mapbox(gps_frame_edited, lat='lat', lon='lon',hover_name="time"
                   ,zoom=18,mapbox_style="carto-positron" ) #Sunsetdark

figgps_edit_line.update_traces(line_color='#d62728')
figgps_edit.add_trace(figgps_edit_line.data[0])
figgps_edit.show()

In [ ]:
gps_frame_edited=get_gps_at_time_metrics(gps_frame, [0])

In [ ]:
import pandas

gps_garmin_pd = pandas.read_csv(filename_gpsgarmin)
#add the new columns to this
gps_garmin_pd["lat std"] = gps_garmin_lat_error*np.ones(len(gps_garmin_pd['lat'].values))
gps_garmin_pd["lon std"] = gps_garmin_lon_error*np.ones(len(gps_garmin_pd['lon'].values))
figgps_garmin = px.scatter_mapbox(gps_garmin_pd, lat='lat', lon='lon',hover_name="time", size='hdop',
                    color_continuous_scale=px.colors.sequential.ice, size_max=gps_garmin_hdop_circle_size, zoom=18,
                  #marker = {'size': 20, 'symbol': ["cross"]},
                  mapbox_style="carto-positron" ) #Sunsetdark
go.Scattermapbox(
marker = {'size': 20, 'symbol': ["cross"]}
)
figgps_garmin_line = px.line_mapbox(gps_garmin_pd, lat='lat', lon='lon',hover_name="time",
                   zoom=18,
                  #marker = {'size': 20, 'symbol': ["cross"]},
                    mapbox_style="carto-positron" ) #Sunsetdark
figgps_garmin.add_trace(figgps_garmin_line.data[0])
figgps_garmin.show()

In [ ]:
#mixed figures
fig_all_gps=figgps_edit
fig_all_gps.add_trace(figgps_garmin.data[0]) # adds the line trace to the first figure
fig_all_gps.add_trace(figgps_garmin.data[1]) # adds the line trace to the first figure
fig_all_gps.show()
fig_all_gps.write_html("gps_all.html")

In [ ]:
import plotly.express as px
figgps_lat = px.scatter(gps_frame_edited, x="time", y="lat",error_y="lat std",title="Latitude mean (line) and standard deviation (bar)").update_layout(
                                                            xaxis_title="Time [s]", yaxis_title="Latitude [decimal]")
figgps_lat2 = px.line(gps_frame_edited, x="time", y="lat",error_y="lat std")
figgps_lat.add_trace(figgps_lat2.data[0]) # adds the line trace to the first figure
figgps_lat.update_traces(line=dict(color='red',width=3))
figgps_lat2 = px.line(gps_garmin_pd, x="time", y="lat")
figgps_lat.add_trace(figgps_lat2.data[0]) # adds the line trace to the first figure                        
figgps_lat2 = px.line(gps_garmin_pd, x="time", y="lat",error_y="lat std")
figgps_lat.add_trace(figgps_lat2.data[0]) # adds the line trace to the first figure
figgps_lat.show()


figgps_lon = px.scatter(gps_frame_edited, x="time", y="lon",error_y="lon std",title="Longitude mean (line) and standard deviation (bar)").update_layout(
                                                            xaxis_title="Time [s]", yaxis_title="Longitude [decimal]")
figgps_lon2 = px.line(gps_frame_edited, x="time", y="lon",error_y="lon std")
figgps_lon.add_trace(figgps_lon2.data[0]) # adds the line trace to the first figure
figgps_lon.update_traces(line=dict(color='red',width=3))
figgps_lon2 = px.line(gps_garmin_pd, x="time", y="lon")
figgps_lon.add_trace(figgps_lon2.data[0]) # adds the line trace to the first figure
figgps_lon2 = px.line(gps_garmin_pd, x="time", y="lon",error_y="lon std")
figgps_lon.add_trace(figgps_lon2.data[0]) # adds the line trace to the first figure     
figgps_lon.show()


figgps_hdop = px.scatter(gps_frame_edited, x="time", y="hdop",error_y="hdop std",title="Horizontal DOP mean (line) and standard deviation (bar)").update_layout(
                                                            xaxis_title="Time [s]", yaxis_title="Horizontal DOP")
figgps_hdop2 = px.line(gps_frame_edited, x="time", y="hdop",error_y="hdop std")
figgps_hdop.add_trace(figgps_hdop2.data[0]) # adds the line trace to the first figure
figgps_hdop.update_traces(line=dict(color='red',width=3))
figgps_hdop2 = px.line(gps_garmin_pd, x="time", y="hdop")
figgps_hdop.add_trace(figgps_hdop2.data[0]) # adds the line trace to the first figure    
figgps_hdop.show()

In [ ]:
 index=7241    
print(byte_array[index]+byte_array[index+1]*256)
print(byte_array[index:(index+70)])
print(byte_array[(index+44):(index+70)])
print(byte_array[index+18:index+22],byte_array[index+22:index+26])

In [ ]:
!pip install cppyy

In [ ]:
import cppyy #pip install cppyy
import cppyy.ll #low level app for using pointers
cppyy.cppdef('''
char* hello4(char* input) {
    input="i has changed the input, muajajaja";
    return("Hello, World!");
}''')
inp="nothing"
print(inp)
test=cppyy.gbl.hello4(inp)   # calls the C++ function 'hello' defined above
print(test)
print(inp)

In [ ]:
import  cppyy #pip install cppyy
import cppyy.ll #low level app for using pointers
#cppyy.include('/libraries/memory_structures/memory_struct_DEFINES.h') # bring in C++ definitions
#cppyy.load_library('/libraries/memory_structures/memory_struct_DEFINES.h') 
cppyy.add_include_path('libraries')#set the path were libraries are localed
cppyy.include('memory_structures/memory_struct_imu0_accgyro.h') # bring in C++ definitions
cppyy.include('memory_structures/memory_struct_imu0_mag.h') # bring in C++ definitions
cppyy.include('memory_structures/memory_struct_gpsneoxm.h') # bring in C++ definitions
cppyy.include('memory_structures/memory_struct_temp.h') # bring in C++ definitions
cppyy.cppdef("""
void* create_float_array(int sz) {
float* pf = (float*)malloc(sizeof(float)*sz);
for (int i = 0; i < sz; ++i) pf[i] = 2*i;
return pf;
}""")
cppyy.cppdef("""
void* create_float_array(int sz) {
float* pf = (float*)malloc(sizeof(float)*sz);
for (int i = 0; i < sz; ++i) pf[i] = 2*i;
return pf;
}""")
import numpy as np
NDATA = 8
arr = cppyy.gbl.create_float_array(NDATA)
print(arr)
arr.reshape((NDATA,)) # adjust the llv's size
v = np.frombuffer(arr, dtype=np.float32, count=NDATA) # cast to float
print(len(v))
print(v)

In [ ]:
import pynmea2
msg = pynmea2.parse("$GPGGA,184353.07,1929.045,S,02410.506,E,1,04,2.6,100.00,M,-33.9,M,,0000*6D")
print(msg.latitude)